# 课后练习2：推荐系统

In [104]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import scipy.io as sio

sns.set(context="notebook", style="white")

### 1. load data and setting up

According to the directions：  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        X: num_movies(1682) $\times$ num_features(10) matrix of movie features  
&nbsp;&nbsp;&nbsp;&nbsp;    Theta: num_user(943) $\times$ num_features(10) matrix of user features  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        Y: num_movies $\times$ num_users matrix of user rating of movies  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        R: num_movies $\times$ num_users matrix, where R(i,j)=1 if the i-th movie was rated by the j-th user (2进制数组)

In [105]:
movies_mat = sio.loadmat('./data/ex8_movies.mat')
Y, R = movies_mat.get('Y'), movies_mat.get('R')

param_mat = sio.loadmat('./data/ex8_movieParams.mat')
theta, X = param_mat.get('Theta'), param_mat.get('X')

Y.shape, R.shape, theta.shape, X.shape

((1682, 943), (1682, 943), (943, 10), (1682, 10))

### 2. 代价函数：$J(x^{(1)},...,x^{(n_m)},\theta^{(1)},...,\theta^{(n_u)})=\frac{1}{2}\sum_\limits{(i,j):r(i,j)=1}((\theta^{(j)})^Tx^{(i)}-y^{(i,j)})^2+\frac{\lambda}{2}\sum_\limits{i=1}^{n_m}\sum_\limits{k=1}^{n}(x_k^{(i)})^2+\frac{\lambda}{2}\sum_\limits{j=1}^{n_u}\sum_\limits{k=1}^{n}(\theta_k^{(j)})^2$

In [106]:
def serialize(X, theta):
    
    return np.concatenate((X.ravel(), theta.ravel()))

def deserialize(param, n_movie, n_user, n_features):
    
    return param[:n_movie * n_features].reshape(n_movie, n_features), param[n_movie * n_features:].reshape(n_user, n_features)

def cost(param, Y, R, n_features):
    
    n_movie, n_user = Y.shape
    X, theta = deserialize(param, n_movie, n_user, n_features)
    
    inner = np.multiply(X @ theta.T - Y, R)
    
    return np.power(inner, 2).sum() / 2

def regularized_cost(param, Y, R, n_features, l=1):
    
    reg_term = np.power(param, 2).sum() * (l / 2)
    
    return cost(param, Y, R, n_features) + reg_term

### 3. 梯度：  
### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$x_k^{(i)}:=x_k^{(i)}-\alpha\left(\sum_{j:r(i,j)=1}(((\theta^{(j)})^Tx^{(i)}-y^{(i,j)})\theta_k^{j}+\lambda x_k^{(i)}\right)$  
### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\theta_k^{(i)}:=\theta_k^{(i)}-\alpha\left(\sum_{i:r(i,j)=1}(((\theta^{(j)})^Tx^{(i)}-y^{(i,j)})x_k^{(i)}+\lambda \theta_k^{(j)}\right)$

In [107]:
def gradient(param, Y, R, n_features):
    
    n_movie, n_user = Y.shape
    X, theta = deserialize(param, n_movie, n_user, n_features)
    
    inner = np.multiply(X @ theta.T - Y, R)
    X_grad = inner @ theta
    theta_grad = inner.T @ X
    
    return serialize(X_grad, theta_grad)

def regularized_gradient(param, Y, R, n_features, l=1):
    
    grad = gradient(param, Y, R, n_features)
    reg_term = l * param
    
    return grad + reg_term

### 4. 运行系统

#### 练习中，给出的计算代价是 22.22，我们试试是不是一致，以检验我们的代价函数是否正确。也可检验下正则化代价，正则化梯度，就不全部检验了。

In [108]:
users = 4
movies = 5
features = 3

X_sub = X[:movies, :features]
theta_sub = theta[:users, :features]
Y_sub = Y[:movies, :users]
R_sub = R[:movies, :users]

param_sub = serialize(X_sub, theta_sub)
cost(param_sub, Y_sub, R_sub, features)

22.224603725685675

#### 接下来，我们按照练习手册中的方法，用上述系统给某个用户推荐电影，最后核对是否与手册中的结果一致

#### 第一步：解析 movie_ids.txt 文件，该文件中是 1682 部电影的名称，我们将所有电影名载入到 movie_list 中

In [109]:
movie_list = []

with open('./data/movie_ids.txt', encoding='latin-1') as f:
    for line in f:
        tokens = line.strip().split(' ')
        movie_list.append(' '.join(tokens[1:]))

movie_list = np.array(movie_list)        # 将存放电影名的列表，转换为 ndarray 对象

In [110]:
movie_list[0]

'Toy Story (1995)'

#### 第二步：虚拟一个用户，该用户仅仅对 1682 部电影中的 11 部进行过评分，练习中已给出

In [111]:
ratings = np.zeros(1682)

ratings[0] = 4
ratings[6] = 3
ratings[11] = 5
ratings[53] = 4
ratings[63] = 5
ratings[65] = 3
ratings[68] = 5
ratings[97] = 2
ratings[182] = 4
ratings[225] = 5
ratings[354] = 5

#### 第三步：准备数据。

In [112]:
Y, R = movies_mat.get('Y'), movies_mat.get('R')
Y.shape, R.shape

((1682, 943), (1682, 943))

In [113]:
Y = np.insert(Y, 0, ratings, axis=1)        # 将虚拟的用户插入到第 0 列
R = np.insert(R, 0, ratings != 0, axis=1)    # 与 Y 一致，但插入的是二进制掩码

Y.shape, R.shape

((1682, 944), (1682, 944))

In [114]:
n_features = 50
n_movie, n_user = Y.shape
l = 10

##### 随机初始化 X 和 theta

In [115]:
X = np.random.standard_normal((n_movie, n_features))
theta = np.random.standard_normal((n_user, n_features))

X.shape, theta.shape

((1682, 50), (944, 50))

In [116]:
param = serialize(X, theta)

##### 归一化评分矩阵

In [117]:
Y_norm = Y - Y.mean()
Y_norm.mean()

4.6862111343939375e-17

#### 第四步：训练模型

In [118]:
import scipy.optimize as opt

In [119]:
res = opt.minimize( fun = regularized_cost,
                    x0 = param,
                    args = (Y_norm, R, n_features, l),
                    method = 'TNC',
                    jac = regularized_gradient
                    )

In [120]:
res

     fun: 64721.49781506433
     jac: array([-2.95446267e-07,  3.95324774e-07,  1.48849110e-07, ...,
       -3.76053618e-07, -3.67190009e-07,  1.43719068e-06])
 message: 'Converged (|f_n-f_(n-1)| ~= 0)'
    nfev: 3037
     nit: 94
  status: 1
 success: True
       x: array([ 0.35131505,  0.12542337,  0.20051368, ..., -0.38833666,
       -0.25677522,  0.194782  ])

##### 我们训练好的最优化的 x 和 theta 都存在 res.x 中，我们将其拆分开

In [121]:
X_trained, theta_trained = deserialize(res.x, n_movie, n_user, n_features)
X_trained.shape, theta_trained.shape

((1682, 50), (944, 50))

##### 对第 $i$ 部电影的评分应加上均值： $X^{(i)}(\theta^{(i)})^{T} + \mu_{i}$

In [122]:
prediction = X_trained @ theta_trained.T
my_pred = prediction[:, 0] + Y.mean()

##### 对第 1 列虚拟用户的预测评分进行排序（默认为从小到大），反向切片后为从大到小

In [123]:
idx = np.argsort(my_pred)[::-1]
idx.shape

(1682,)

##### 按照掩码 idx 索引，取出前 10 部电影

In [124]:
my_pred[idx][:10]

array([4.12534772, 4.0441397 , 3.99324422, 3.91902781, 3.81690928,
       3.81556206, 3.76602658, 3.76322808, 3.75905655, 3.75077471])

In [125]:
for m in movie_list[idx][:10]:
    print(m)

Titanic (1997)
Star Wars (1977)
Shawshank Redemption, The (1994)
Forrest Gump (1994)
Raiders of the Lost Ark (1981)
Braveheart (1995)
Return of the Jedi (1983)
Usual Suspects, The (1995)
Godfather, The (1972)
Schindler's List (1993)
